# Gaussian Mixture Model

In [ ]:
import os
import sys
import time
import pickle

import pandas as pd
import numpy as np
import tensorflow as tf

from collections import namedtuple

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import config

# USE DOUBLE PRECISION FLOATING POINT: `tf.float64`
#config.TF_FLOAT = tf.float64
#config.TF_INT = tf.int64
#config.NP_FLOAT = np.float64

In [ ]:
from loggers.train_logger import TrainLogger
from loggers.run_logger import RunLogger
import utils.file_io as io

import matplotlib as mpl
import matplotlib.pyplot as plt

%autoreload 2
%matplotlib notebook
np.set_printoptions(precision=5)
np.set_printoptions(suppress=True)

from utils.distributions import GMM, gen_ring
from dynamics.dynamics import Dynamics

from models.gmm_model import GaussianMixtureModel

In [ ]:
def create_session(config, checkpoint_dir, monitored=False):
    if monitored:
        sess_kwargs = {
            'checkpoint_dir': checkpoint_dir,
            'hooks': [],
            'config': config,
            'save_summaries_secs': None,
            'save_summaries_steps': None,
        }
        return tf.train.MonitoredTrainingSession(**sess_kwargs)
    
    return tf.Session(config=config)

In [ ]:
def draw_ellipse(position, covariance, ax=None, **kwargs):
    """Draw an ellipse with a given position and covariance"""
    ax = ax or plt.gca()
    
    # Convert covariance to principal axes
    if covariance.shape == (2, 2):
        U, s, Vt = np.linalg.svd(covariance)
        angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
        width, height = 2 * np.sqrt(s)
    else:
        angle = 0
        width, height = 2 * np.sqrt(covariance)
    
    # Draw the Ellipse
    for nsig in range(1, 4):
        ax.add_patch(
            Ellipse(position, nsig * width, nsig * height, angle, **kwargs)
        )
        
def plot_gmm(gmm, X, label=True, ax=None):
    ax = ax or plt.gca()
    labels = gmm.fit(X).predict(X)
    if label:
        ax.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis', zorder=2)
    else:
        ax.scatter(X[:, 0], X[:, 1], s=40, zorder=2)
    ax.axis('equal')
    
    w_factor = 0.2 / gmm.weights_.max()
    for pos, covar, w in zip(gmm.means_, gmm.covars_, gmm.weights_):
        draw_ellipse(pos, covar, alpha=w * w_factor)

In [ ]:
def save_inference_data(samples, px, out_dir, type_str='l2hmc'):
    px = np.array(px)
    samples = np.array(samples)
    all_samples = samples.reshape((-1, 2))
    x, y = all_samples.T
    xmean, ymean = x.mean(), y.mean()
    
    type_str = type_str.upper()
    shape_str = f'{type_str} samples.shape: {samples.shape}\n'
    _str = f'{type_str} Averages (x_avg, y_avg): ({xmean:.5g}, {ymean:.5g})'
    print(shape_str + _str)
    
    samples_out_file = os.path.join(out_dir, f'{type_str}_inference_samples.pkl')
    px_out_file = os.path.join(out_dir, f'{type_str}_inference_probs.pkl')
    
    print(f'Saving samples to: {samples_out_file}')
    with open(samples_out_file, 'wb') as f:
        pickle.dump(samples, f)
        
    print(f'Saving probs to: {px_out_file}')
    with open(px_out_file, 'wb') as f:
        pickle.dump(px, f)
        
    means_file = os.path.join(out_dir, f'{type_str}_inference_means.txt')
    with open(means_file, 'w') as f:
        _ = f.write(shape_str)
        _ = f.write(_str)

#### Setup paramters for `GaussianMixtureModel`

In [ ]:
from params.gmm_params import GMM_PARAMS
from loggers.train_logger import TrainLogger
from main import train_setup, create_config
from update import set_precision

params = GMM_PARAMS
#params['float64'] = True
#set_precision('float64')

root_dir = 'gmm_logs'
params, hooks = train_setup(params, log_file=None,
                            root_dir=root_dir,
                            run_str=False)
params['beta_init'] = 1. / 20.
params['diag'] = True
params['center'] = 1.5
params['sigma1'] = 0.02
params['sigma2'] = 0.06
params['num_steps'] = 10
params['eps'] = 0.2
params['batch_size'] = 128

# Whether or not to use Gaussian loss instead of the usual ESJD loss
params['use_gaussian_loss'] = False
params['loss_scale'] = 1.

_ = params.pop('sigma')
_ = params.pop('zero_translation')
_ = [print(f'{k}: {v}') for k, v in params.items()]

#### Build `GaussianMixtureModel`

In [ ]:
model = GaussianMixtureModel(params)
target_samples = model.distribution.get_samples(int(2e6))

#### Plot target distribution

In [ ]:
from plotters.plot_utils import _gmm_plot
figs_dir = os.path.join(model.log_dir, 'figures')
io.check_else_make_dir(figs_dir)
out_file = os.path.join(figs_dir, 'target_distribution.pdf')

kwargs = {
    'out_file': out_file,
    'fill': False,
    'ellipse': False,
    'title': 'Target distribution of GMM',
    'ls': '',
    'axis_scale': 'scaled',
}

fig, axes = plt.subplots()
ax = _gmm_plot(model.distribution, target_samples, **kwargs)
xlim_ = ax.get_xlim()
xmin, xmax = -0.7, 2.1
ymin, ymax = -0.5, 2.1
ax.set_xlim((xmin, xmax))
ax.set_ylim((ymin, ymax))

In [ ]:
import scipy.stats as st
from mpl_toolkits.mplot3d import Axes3D

_samples = target_samples[:10000]
#xlims, ylims = get_lims(_samples)
#xmin, xmax = -0.8, 2.75
#ymin, ymax = -0.8, 2.75
#xmin, xmax = -0.75, 1.5
#ymin, ymax = -0.5, 1.5

# Create meshgrid
xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]

x = _samples[:, 0]
y = _samples[:, 1]

positions = np.vstack([xx.ravel(), yy.ravel()])
values = np.vstack([x, y])
kernel = st.gaussian_kde(values)
f = np.reshape(kernel(positions).T, xx.shape)

In [ ]:
cmap = 'coolwarm'
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_surface(xx, yy, f, rstride=1, cstride=1,
                       cmap=cmap, edgecolor='none', alpha=0.6)
# Plot projections of the contours for each dimension.  By choosing offsets
# that match the appropriate axes limits, the projected contours will sit on
# the 'walls' of the graph
cset = ax.contour(xx, yy, f, zdir='z', offset=-0.1, cmap=cmap)
cset = ax.contour(xx, yy, f, zdir='x', offset=xmin, cmap=cmap)
cset = ax.contour(xx, yy, f, zdir='y', offset=ymax, cmap=cmap)
#_ = ax.axis('equal')
xlim = [xmin, xmax]
ylim = [ymin, ymax]
_ = ax.set_xlim(xlim)
_ = ax.set_ylim(ylim)
_ = ax.set_xlabel('x')
_ = ax.set_ylabel('y')

z_lim = ax.get_zlim()
_ = ax.set_zlim((-0.1, z_lim[1]))

#### Calculate the "average" mean using samples drawn from target distribution

\begin{align*}
\vec{\mu}_{p} &= \sum_{i} \vec{x}_{i},\\ 
    \mathrm{where:}\quad \vec{x} &\sim \pi_{1}\, \mathcal{N}\left(\vec{\mu}_{1}, \vec{\Sigma}_{2}\right) 
        + \pi_2\, \mathcal{N}\left(\vec{\mu}_{2}, \vec{\Sigma}_{2}\right)
\end{align*}

In [ ]:
from scipy.stats import sem

samples_avg = target_samples.mean(axis=0) # get average (x, y) for each chain
_x, _y = samples_avg.T

target_samples_shape_str = f'target_samples.shape: {target_samples.shape}\n'
target_samples_str = f'(x_avg, y_avg): ({_x.mean():.6g}, {_y.mean():.6g})\n'

print(target_samples_shape_str + target_samples_str)

target_means_file = os.path.join(model.log_dir, 'means.txt')
with open(target_means_file, 'w') as f:
    _ = f.write(target_samples_shape_str + target_samples_str)

#### Create session and `train_logger`

In [ ]:
train_logger = TrainLogger(model, params['log_dir'],
                           logging_steps=10, summaries=params['summaries'])

config, params = create_config(params)

checkpoint_dir = os.path.join(model.log_dir, 'checkpoints')
io.check_else_make_dir(checkpoint_dir)

sess = create_session(config, checkpoint_dir, monitored=True)
tf.keras.backend.set_session(sess)

#### Reversibility check
Check reversibility using `model._check_reversibility()` method and compute the difference between `(x_init, v_init)` and `backward(forward(x_init, v_init))`

In [ ]:
# -------------------------------------------------------------------
# Check reversibility using `model._check_reversibility()` method
# -------------------------------------------------------------------
samples_init = np.random.randn(*model.x.shape)
feed_dict = {
    model.x: samples_init,
    model.beta: 1.,
    model.net_weights[0]: 1.,
    model.net_weights[1]: 1.,
    model.net_weights[2]: 1.,
    model.train_phase: False
}

x_diff, v_diff = sess.run([model.x_diff, model.v_diff], feed_dict=feed_dict)
x_diff, v_diff
# (3.9827573e-06, 0.00016799556)
# (2.4846453e-05, 0.00034340395)

### Training
Train `GaussianMixtureModel` using `GaussianMixtureModelTrainer`.

In [ ]:
from trainers.gmm_trainer import GaussianMixtureModelTrainer
trainer = GaussianMixtureModelTrainer(sess, model, logger=train_logger)

train_kwargs = {
    'samples_np': np.random.randn(*model.x.shape),
    'beta_np': model.beta_init,
    'net_weights': [1., 1., 1.],
    'print_steps': 1.,
}

trainer.train(5000, **train_kwargs)

### Inference

Load and restore a trained model for running inference, using `GaussianMixtureModelRunner` and `RunLogger` for generating summaries and saving statistics.

In [ ]:
sess.close()
tf.reset_default_graph()

In [ ]:
checkpoint_dir = os.path.join(model.log_dir, 'checkpoints')
checkpoint_file = tf.train.latest_checkpoint(checkpoint_dir)
config = tf.ConfigProto()
sess = tf.Session(config=config)
saver = tf.train.import_meta_graph(f'{checkpoint_file}.meta')
saver.restore(sess, checkpoint_file)

In [ ]:
from loggers.run_logger import RunLogger
from runners.gmm_runner import GaussianMixtureModelRunner

run_ops = tf.get_collection('run_ops')
inputs = tf.get_collection('inputs')

run_logger = RunLogger(params, inputs, run_ops,
                       save_lf_data=False,
                       model_type='gmm_model')

runner = GaussianMixtureModelRunner(sess, params, inputs,
                                    run_ops, logger=run_logger)

#### Run L2HMC inference using trained model

In [ ]:
kwargs = {
    'run_steps': 10000,
    'beta': 1.,
    'net_weights': [1., 1., 1.],
    'eps': runner.eps,
}
run_str = run_logger._get_run_str(**kwargs)
kwargs['run_str'] = run_str

run_logger.reset(**kwargs)
runner.run(**kwargs)

samples_out = np.array(run_logger.samples_arr)
samples_out.shape

In [ ]:
kwargs = {
    'run_steps': 10000,
    'beta': 1.,
    'net_weights': [1., 1., 1.],
    'eps': runner.eps,
}
run_str = run_logger._get_run_str(**kwargs)
kwargs['run_str'] = run_str

run_logger.reset(**kwargs)
runner.run(**kwargs)

samples_out1 = np.array(run_logger.samples_arr)
samples_out1.shape

In [ ]:
samples_out = np.array(run_logger.samples_arr)
px_out = np.array(run_logger.px_arr)
out_dir = run_logger.run_dir
save_inference_data(samples_out, px_out, out_dir, type_str='L2HMC')

In [ ]:
from plotters.plot_utils import _gmm_plot, get_lims
figs_dir = os.path.join(model.log_dir, 'figures')
io.check_else_make_dir(figs_dir)
out_file = os.path.join(figs_dir, 'single_l2hmc_chain.pdf')

kwargs = {
    'out_file': out_file,
    'fill': False,
    'ellipse': False,
    'title': 'Single L2HMC chain',
    'ls': '-',
    'axis_scale': 'scaled',
}

fig, axes = plt.subplots()
ax = _gmm_plot(model.distribution, samples_out[:, 42], **kwargs)
xlim, ylim = get_lims(model.distribution.get_samples(500))
#xlim_ = ax.get_xlim()
#xmin, xmax = -0.7, 2.1
#ymin, ymax = -0.5, 2.1
ax.set_xlim(xlim)
ax.set_ylim(ylim)

In [ ]:
angles = np.array([0., 2 * np.pi, - np.pi, 3 * np.pi])
coords = [(np.cos(x), np.sin(x)) for x in angles]

In [ ]:
def wrap_angles(angles):
    x, y = np.cos(angles), np.sin(angles)
    angles = np.arctan2(x, y)
    
    return angles

In [ ]:
angles_out = wrap_angles(samples_out)

In [ ]:
angles_out.shape

In [ ]:
angle

In [ ]:
lims = get_lims(model.distribution.get_samples(500))

In [ ]:
all_samples = samples_out.reshape((-1, 2))

X, Y = all_samples[:, 0], all_samples[:, 1]
X.shape, Y.shape

In [ ]:
Xrs = block_resampling(X, 64)
Xrs_ = np.array(Xrs)
Xrs_.shape

In [ ]:
X_avg, X_err = calc_avg_vals_errors(X, num_blocks=64)
Y_avg, Y_err = calc_avg_vals_errors(Y, num_blocks=64)

print(f'({X_avg}, {Y_avg}) +/- ({X_err}, {Y_err})')

In [ ]:
print(f'({X_avg}, {Y_avg}) +/- ({X_err}, {Y_err})')

In [ ]:
samples_outT = samples_out.transpose((1, 0, 2))
samples_outT.shape

xT, yT = samples_outT[:, :, 0], samples_outT[:, :, 1]
xT.shape

In [ ]:
xT_rs = np.array([block_resampling(xx, 20) for xx in xT])
xT_rs.shape

In [ ]:
x_rs = np.array([
    block_resampling(samples_out[:, idx, 0], 20) for idx in samples_out.shape[1]
])

In [ ]:
samples_out_rs = block_resampling(samples_out[:, 0, 0], 20)

In [ ]:
samples_out_rs[0].shape

In [ ]:
np.array(samples_out_rs).shape

In [ ]:
samples_out.shape

In [ ]:
xmu = samples_out[:, :, 0].mean(axis=0)
xmu.shape

In [ ]:
from utils.data_utils import (block_resampling, jackknife, jackknife_err,
                              jackknife_var, calc_avg_vals_errors)

ymu = samples_out[:, :, 1].mean(axis=0)
_x_avg, _x_err = calc_avg_vals_errors(xmu, num_blocks=64)
_y_avg, _y_err = calc_avg_vals_errors(ymu, num_blocks=64)

print(f'Mean +/- err:\n ({_x_avg}, {_y_avg}) +/- ({_x_err}, {_y_err})')

In [ ]:
from plotters.plot_utils import gmm_plot
gaussian_loss = model.use_gaussian_loss
if gaussian_loss:
    _loss_type = 'Gaussian'
else:
    _loss_type = 'Original'

figs_dir = os.path.join(model.log_dir, 'figures')
io.check_else_make_dir(figs_dir)

samples_out = np.array(samples_out)

l2hmc_kwargs = {
    'nrows': 3,
    'ncols': 3,
    'num_points': 2500,
    'ellipse': False,
    'title': f'L2HMC ({_loss_type} loss)',
    'out_file': os.path.join(figs_dir, 'inference_plot.pdf'),
    'axis_scale': 'equal',
}
fig, axes = gmm_plot(model.distribution, samples_out, **l2hmc_kwargs)

In [ ]:
fig.savefig(os.path.join(figs_dir, 'inference_plot1,.pdf'), bbox_inches='tight')

In [ ]:
import scipy.stats as st
from mpl_toolkits.mplot3d import Axes3D

l2hmc_samples = samples_out.reshape((-1, 2))
_samples = l2hmc_samples[:10000]

# Create meshgrid
xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]

x = _samples[:, 0]
y = _samples[:, 1]

positions = np.vstack([xx.ravel(), yy.ravel()])
values = np.vstack([x, y])
kernel = st.gaussian_kde(values)
f = np.reshape(kernel(positions).T, xx.shape)

In [ ]:
cmap = 'coolwarm'
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_surface(xx, yy, f, rstride=1, cstride=1,
                       cmap=cmap, alpha=0.8)  # edgecolor='none')
# Plot projections of the contours for each dimension.  By choosing offsets
# that match the appropriate axes limits, the projected contours will sit on
# the 'walls' of the graph
cset = ax.contour(xx, yy, f, zdir='z', offset=-0.2, cmap=cmap)
cset = ax.contour(xx, yy, f, zdir='x', offset=xmin, cmap=cmap)
cset = ax.contour(xx, yy, f, zdir='y', offset=ymax, cmap=cmap)
#_ = ax.axis('equal')
xlim = [xmin, xmax]
ylim = [ymin-0.05, ymax+0.05]
_ = ax.set_xlim(xlim)
_ = ax.set_ylim(ylim)
_ = ax.set_xlabel('x')
_ = ax.set_ylabel('y')

z_lim = ax.get_zlim()
_ = ax.set_zlim((-0.1, z_lim[1]))

#### RUN GENERIC HMC:

In [ ]:
hmc_kwargs = {
    'run_steps': 5000,
    'beta': 1.,
    'net_weights': [0., 0., 0.],
    'eps': runner.eps,
}

run_str = run_logger._get_run_str(**hmc_kwargs)
hmc_kwargs['run_str'] = run_str

run_logger.reset(**hmc_kwargs)

In [ ]:
runner.run(**hmc_kwargs)

samples_out_hmc = np.array(run_logger.samples_arr)
px_out_hmc = np.array(run_logger.px_arr)

inference_dir = os.path.join(model.log_dir, 'inference')
io.check_else_make_dir(inference_dir)
save_inference_data(samples_out_hmc, px_out_hmc, inference_dir, type_str='HMC')

In [ ]:
from plotters.plot_utils import gmm_plot


figs_dir = os.path.join(model.log_dir, 'figures')
io.check_else_make_dir(figs_dir)

hmc_plt_kwargs = {
    'nrows': 3,
    'ncols': 3,
    'num_points': 5000,
    'ellipse': False,
    'title': 'HMC (Gaussian losss)',
    'out_file': os.path.join(figs_dir, 'hmc_inference_plot.pdf')
}
fig, axes = gmm_plot(model.distribution,
                     samples_out_hmc,
                     **hmc_plt_kwargs)

In [ ]:
import scipy.stats as st
from mpl_toolkits.mplot3d import Axes3D

hmc_samples = samples_out_hmc.reshape((-1, 2))
_samples = hmc_samples[:10000]

# Create meshgrid
xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]

x = _samples[:, 0]
y = _samples[:, 1]

positions = np.vstack([xx.ravel(), yy.ravel()])
values = np.vstack([x, y])
kernel = st.gaussian_kde(values)
f = np.reshape(kernel(positions).T, xx.shape)

In [ ]:
cmap = 'coolwarm'
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_surface(xx, yy, f, rstride=1, cstride=1,
                       cmap=cmap, edgecolor='none', alpha=0.8)
# Plot projections of the contours for each dimension.  By choosing offsets
# that match the appropriate axes limits, the projected contours will sit on
# the 'walls' of the graph
cset = ax.contour(xx, yy, f, zdir='z', offset=-0.1, cmap=cmap)
cset = ax.contour(xx, yy, f, zdir='x', offset=xmin, cmap=cmap)
cset = ax.contour(xx, yy, f, zdir='y', offset=ymax, cmap=cmap)
#_ = ax.axis('equal')
#xlim = [xmin, xmax]
#ylim = [ymin-0.05, ymax+0.05]
#_ = ax.set_xlim(xlim)
#_ = ax.set_ylim(ylim)
_ = ax.set_xlabel('x')
_ = ax.set_ylabel('y')

z_lim = ax.get_zlim()
_ = ax.set_zlim((-0.1, z_lim[1]))

### Extra

In [ ]:
from loggers.run_logger import RunLogger
from runners.gmm_runner import GaussianMixtureModelRunner

run_ops = tf.get_collection('run_ops')
inputs = tf.get_collection('inputs')

run_logger = RunLogger(params, inputs, run_ops,
                       save_lf_data=False,
                       model_type='gmm_model')
runner = GaussianMixtureModelRunner(sess, params, inputs,
                                    run_ops, logger=run_logger)
kwargs = {
    'run_steps': 5000,
    'beta': 1.,
    'net_weights': [1., 1., 1.],
}
runner.run(**kwargs)

samples_out = np.array(run_logger.samples_arr)
samples_out.shape

from plotters.plot_utils import gmm_plot

figs_dir = os.path.join(model.log_dir, 'figures')
io.check_else_make_dir(figs_dir)

l2hmc_kwargs = {
    'nrows': 3,
    'ncols': 3,
    'num_points': 250,
    'ellipse': False,
    'title': 'L2HMC (Gaussian losss)',
    'out_file': os.path.join(figs_dir, 'inference_plot.pdf')
}
fig, axes = gmm_plot(model.distribution, samples_out, **l2hmc_kwargs)

In [ ]:
samples_out = []
px_out = []

feed_dict = {
    model.x: None,
    model.beta: 1.,
    model.net_weights[0]: 1.,
    model.net_weights[1]: 1.,
    model.net_weights[2]: 1.,
    model.train_phase: False
}
samples = np.random.randn(*model.x.shape)
num_steps = 5000
for i in range(num_steps):
    t0 = time.time()
    feed_dict.update({
        model.x: samples,
    })
    samples, px = sess.run([model.x_out, model.px], feed_dict=feed_dict)
    _px = np.mean(px)
    samples_out.append(samples)
    px_out.append(_px)
    print(f'{i}/{num_steps}, px: {_px:^.4g}, t/step: {time.time() - t0:^.4g}')

samples_out = np.array(samples_out)
all_samples = samples_out.reshape((-1, 2))
x_, y_ = all_samples.T
xmean, ymean = x_.mean(), y_.mean()

In [ ]:
samples_out_hmc = []
px_out_hmc = []

feed_dict = {
    model.x: None,
    model.beta: 1.,
    model.net_weights[0]: 0.,
    model.net_weights[1]: 0.,
    model.net_weights[2]: 0.,
    model.train_phase: False
}
samples = np.random.randn(*model.x.shape)
num_steps = 5000
for i in range(num_steps):
    t0 = time.time()
    feed_dict.update({
        model.x: samples,
    })
    samples, px = sess.run([model.x_out, model.px], feed_dict=feed_dict)
    _px = np.mean(px)
    samples_out_hmc.append(samples)
    px_out_hmc.append(_px)
    print(f'{i}/{num_steps}, px: {_px:^.4g}, t/step: {time.time() - t0:^.4g}')
    
samples_out_hmc = np.array(samples_out_hmc)
all_samples_hmc = samples_out_hmc.reshape((-1, 2))
x_hmc, y_hmc = all_samples_hmc.T
xmean_hmc, ymean_hmc = x_hmc.mean(), y_hmc.mean()

In [ ]:
inference_dir = os.path.join(model.log_dir, 'inference')
io.check_else_make_dir(inference_dir)

l2hmc_shape_str = f'L2HMC samples.shape: {samples_out.shape}\n'
hmc_shape_str = f'HMC samples.shape: {samples_out_hmc.shape}\n'
l2hmc_str = f'L2HMC Averages (x_avg, y_avg): ({xmean:.5g}, {ymean:.5g})\n'
hmc_str = f'HMC Averages (x_avg, y_avg): ({xmean_hmc:.5g}, {ymean_hmc:.5g})\n'

print(l2hmc_shape_str + l2hmc_str)
print('\n')
print(hmc_shape_str + hmc_str)

samples_out_file = os.path.join(inference_dir, 'inference_samples.pkl')
px_out_file = os.path.join(inference_dir, 'inference_probs.pkl')
with open(samples_out_file, 'wb') as f:
    pickle.dump(samples_out, f)
with open(px_out_file, 'wb') as f:
    pickle.dump(px_out, f)
    
samples_out_file_hmc = os.path.join(inference_dir, 'hmc_samples.pkl')
px_out_file_hmc = os.path.join(inference_dir, 'hmc_probs.pkl')
with open(samples_out_file_hmc, 'wb') as f:
    pickle.dump(samples_out_hmc, f)
with open(px_out_file_hmc, 'wb') as f:
    pickle.dump(px_out_hmc, f)

inference_means_file = os.path.join(inference_dir, 'inference_means.txt')
with open(inference_means_file, 'w') as f:
    _ = f.write(l2hmc_shape_str)
    _ = f.write(l2hmc_str)
    _ = f.write(20 * '-' + '\n')
    _ = f.write(hmc_shape_str)
    _ = f.write(hmc_str)

In [ ]:
all_samples.shape

In [ ]:
from plotters.plot_utils import _gmm_plot

fig, axes = plt.subplots()
ax = _gmm_plot(model.distribution, all_samples[:10000], ellipse=True)
_ = ax.axis('equal')

In [ ]:
samples_out_r = samples_out[:, ::-1]

In [ ]:
from plotters.plot_utils import gmm_plot

l2hmc_kwargs = {
    'nrows': 5,
    'ncols': 4,
    'num_points': 5000,
    'title': 'L2HMC (Gaussian losss)',
    'out_file': os.path.join(figs_dir, 'inference_plot.pdf'),
    'ellipse': True,
    'num_contours': 4,
    'cmap': None,
    'fill': True,
}
fig, axes = gmm_plot(model.distribution, samples_out_r, **l2hmc_kwargs)

In [ ]:
import scipy.stats as st
from mpl_toolkits.mplot3d import Axes3D

#_samples = samples_out_r[:5000, 0]
_samples = all_samples[:10000]
#xlims, ylims = get_lims(_samples)
xmin, xmax = [-1., 3.]
ymin, ymax = [-1., 3.]

# Create meshgrid
xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]

x = _samples[:, 0]
y = _samples[:, 1]

positions = np.vstack([xx.ravel(), yy.ravel()])
values = np.vstack([x, y])
kernel = st.gaussian_kde(values)
z = np.reshape(kernel(positions).T, xx.shape)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_surface(xx, yy, z, rstride=1, cstride=1,
                       cmap='coolwarm', edgecolor='none', alpha=0.5)
# Plot projections of the contours for each dimension.  By choosing offsets
# that match the appropriate axes limits, the projected contours will sit on
# the 'walls' of the graph
cset = ax.contour(xx, yy, z, zdir='z', offset=-0.1, cmap='coolwarm')
cset = ax.contour(xx, yy, z, zdir='x', offset=xmin, cmap='coolwarm')
cset = ax.contour(xx, yy, z, zdir='y', offset=ymax, cmap='coolwarm')
#_ = ax.axis('equal')
ax.set_xlim((xmin, xmax))
ax.set_ylim((ymin, ymax))
ax.set_xlabel('x')
ax.set_ylabel('y')

z_lim = ax.get_zlim()
ax.set_zlim((-0.1, z_lim[1]))

In [ ]:
from plotters.plot_utils import gmm_plot

hmc_kwargs = {
    'nrows': 3,
    'ncols': 3,
    'num_points': 500,
    'title': 'HMC (Gaussian losss)',
    'out_file': os.path.join(figs_dir, 'hmc_plot.pdf'),
    'ellipse': False,
    'num_contours': 4,
    'cmap': None,
    'fill': True,
}
fig, axes = gmm_plot(model.distribution, samples_out_hmc, **hmc_kwargs)

In [ ]:
import scipy.stats as st

xlims, ylims = get_lims(samples_out[:, 0])
xmin, xmax = xlims
ymin, ymax = ylims

# Create meshgrid
xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]

x = samples_out[:, 0, 0]
y = samples_out[:, 0, 1]

positions = np.vstack([xx.ravel(), yy.ravel()])
values = np.vstack([x, y])
kernel = st.gaussian_kde(values)
f = np.reshape(kernel(positions).T, xx.shape)

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')


surf = ax.plot_surface(xx, yy, f, rstride=1, cstride=1,
                       cmap='coolwarm', edgecolor='none')#alpha=0.3)
#ax.plot_surface(X, Y, Z, rstride=8, cstride=8, alpha=0.3)

# Plot projections of the contours for each dimension.  By choosing offsets
# that match the appropriate axes limits, the projected contours will sit on
# the 'walls' of the graph
#cset = ax.contour(xx, yy, f, zdir='z', offset=-1, cmap='coolwarm')
#cset = ax.contour(xx, yy, f, zdir='x', offset=-2, cmap='coolwarm')
#cset = ax.contour(xx, yy, f, zdir='y', offset=-2, cmap='coolwarm')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('PDF')
#ax.set_title('Surface plot of Gaussian 2D KDE')
#fig.colorbar(surf, shrink=0.5, aspect=5) # add color bar indicating the PDF
ax.view_init(60, 35)

## OLD

In [ ]:
plt.colormaps()

In [ ]:
from matplotlib.patches import Ellipse
from plotters.plot_utils import draw_ellipse
#target_samples = model.distribution.get_samples(5000)
figs_dir = os.path.join(model.log_dir, 'figures')
io.check_else_make_dir(figs_dir)
#y_lims = 1. * np.array([-0.7, 1.35])
nrows = ncols = 2
fig, axes = plt.subplots(nrows=nrows, ncols=ncols)#, sharex=True, sharey=True)
#x_lims=[-0.5, 1.6]
#y_lims=[-1., 1.45]
mus = model.distribution.mus
sigmas = model.distribution.sigmas
pis = model.distribution.pis
#xmin = ymin = np.min(mus) - 4 * np.max(sigmas)
#xmax = ymax = np.max(mus) + 4 * np.max(sigmas)
#xlims = [xmin, xmax]
#ylims = [ymin, ymax]
#  xmin = - np.min(mus) - 5 * np.max(sigmas)

#xlims = [-2, 2]
#ylims = [-2, 2]
#plt.subplots_adjust(hspace=0.25, wspace=0.25)
_idx = 0
for idx in range(nrows):
    for jdx in range(ncols):
        #_ = plot_gaussian_contours(mus, covs,
        #                           xlims=xlims,
        #                           ylims=ylims,
        #                           res=200,
        #                           ax=axes[idx, jdx])
        w_factor = 0.2 / np.max(pis)
        for pos, covar, w in zip(mus, sigmas, pis):
            _ = draw_ellipse(pos, covar,
                             ax=axes[idx, jdx],
                             #alpha = 1.,
                             alpha=w * w_factor,
                             fill=True, lw=1.5)

        #xlims, ylims = get_lims(samples_out[:, _idx])
        #_ = plot_gaussian_contours(mus, sigmas, ax=axes[idx, jdx],
        #                           xlims=xlims, ylims=ylims)
        
        _ = axes[idx, jdx].plot(samples_out[:100, _idx, 0],
                                samples_out[:100, _idx, 1],
                                marker=',', color='gray', alpha=0.4, ls='-',
                                zorder=2)
        _ = axes[idx, jdx].plot(samples_out[:50, _idx, 0],
                                samples_out[:50, _idx, 1],
                                marker=',', color='k', alpha=0.6, ls='',
                                zorder=2)
        _ = axes[idx, jdx].axis('equal')
        #_ = axes[idx, jdx].set_xlim(xlims)
        #_ = axes[idx, jdx].set_ylim(ylims)
        _ = axes[idx, jdx].set_xticks([])
        _ = axes[idx, jdx].set_yticks([])
        #_ = axes[idx, jdx].autoscale()
        _idx += 1
        
_ = axes[0, 0].set_yticks(model.distribution.mus[0])
_ = axes[0, 0].set_yticklabels([str(i) for i in model.distribution.mus[0]])
_ = axes[-1, -1].set_xticks(model.distribution.mus[1])
_ = axes[-1, -1].set_xticklabels([str(i) for i in model.distribution.mus[1]])
_ = fig.suptitle('L2HMC Chains (ESJD loss)')

In [ ]:
import matplotlib as mpl

In [ ]:
mpl.cm.viridis

In [ ]:
for i in range(1, 4):
    print(i)

In [ ]:
idxs = np.linspace(0.1, 0.75, 4)
idxs

In [ ]:
ax.add_patch(Ellipse())

In [ ]:
# Generate some data
from sklearn.datasets.samples_generator import make_blobs
X, y_true = make_blobs(n_samples=400, centers=4,
                       cluster_std=0.60, random_state=0)
X = X[:, ::-1] # flip axes for better plotting
X.shape

In [ ]:
from sklearn import mixture

In [ ]:
gmm = mixture.GMM

In [ ]:
import sklearn

In [ ]:
sklearn.mixture.GaussianMixture()

In [ ]:
gmm = mixture.GMM()

In [ ]:
ax.set_x

In [ ]:
from plotters.plot_utils import gmm_plot

figs_dir = os.path.join(model.log_dir, 'figures')
io.check_else_make_dir(figs_dir)
xlims = [-1, 2]
ylims = [-0.5, 2]

l2hmc_kwargs = {
    'nrows': 3,
    'ncols': 3,
    'num_points': 500,
    'out_file': None,
    'title': 'L2HMC (Gaussian losss)',
    'out_file': os.path.join(figs_dir, 'inference_plot.pdf'),
    'xlims': xlims,
    'ylims': ylims
}
fig, axes = gmm_plot(model.distribution, samples_out, **l2hmc_kwargs)

In [ ]:
plt.contour()

## RESTORE FROM CHECKPOINT

In [ ]:
sess.close()
tf.reset_default_graph()

In [ ]:
checkpoint_dir = os.path.join(model.log_dir, 'checkpoints')
checkpoint_file = tf.train.latest_checkpoint(checkpoint_dir)
config = tf.ConfigProto()
sess = tf.Session(config=config)
saver = tf.train.import_meta_graph(f'{checkpoint_file}.meta')
saver.restore(sess, checkpoint_file)

In [ ]:
run_ops = tf.get_collection('run_ops')
inputs = tf.get_collection('inputs')

from loggers.run_logger import RunLogger
run_logger = RunLogger(params, inputs, run_ops,
                       save_lf_data=False, model_type='gmm_model')

from runners.gmm_runner import GaussianMixtureModelRunner
runner = GaussianMixtureModelRunner(sess, params, inputs,
                                    run_ops, logger=run_logger)

kwargs = {
    'run_steps': 5000,
    'beta': 1.,
    'net_weights': [1., 1., 1.],
}
runner.run(**kwargs)

samples_out = np.array(run_logger.samples_arr)
samples_out.shape

from plotters.plot_utils import gmm_plot

figs_dir = os.path.join(model.log_dir, 'figures')
io.check_else_make_dir(figs_dir)

l2hmc_kwargs = {
    'nrows': 3,
    'ncols': 3,
    'num_points': 250,
    'ellipse': False,
    'title': 'L2HMC (Gaussian losss)',
    'out_file': os.path.join(figs_dir, 'inference_plot.pdf')
}
fig, axes = gmm_plot(model.distribution, samples_out, **l2hmc_kwargs)

In [ ]:
with op

In [ ]:
run_logger.run_data.keys()

In [ ]:
feed_dict = {
    run_logger.inputs_dict['x']: np.random.randn(*(params['batch_size'], params['x_dim'])),
    run_logger.inputs_dict['beta']: 1.,
    run_logger.inputs_dict['net_weights'][0]: 1.,
    run_logger.inputs_dict['net_weights'][1]: 1.,
    run_logger.inputs_dict['net_weights'][2]: 1.,
    run_logger.inputs_dict['train_phase']: False,
}
l2hmc_fns_f = sess.run(run_logger.run_ops_dict['fns_out_f'], feed_dict=feed_dict)

In [ ]:
def extract_l2hmc_fns(fns):
    fnsT = np.transpose(fns, axes=[2, 1, 0, 3, 4])

    out_fns = {}
    names = ['scale', 'translation', 'transformation']
    subnames = ['v1', 'x1', 'x2', 'v2']
    for idx, name in enumerate(names):
        out_fns[name] = {}
        for subidx, subname in enumerate(subnames):
            out_fns[name][subname] = fnsT[idx][subidx]

    return out_fns

In [ ]:
l2hmc_fns_f_dict = extract_l2hmc_fns(l2hmc_fns_f)

In [ ]:
l2hmc_fns_f_dict.keys()

In [ ]:
scale_fns_f = l2hmc_fns_f_dict['scale']
scale_fns_f.keys()

In [ ]:
translation_fns_f = l2hmc_fns_f_dict['translation']
transformation_fns_f = l2hmc_fns_f_dict['transformation']

In [ ]:
translation_fns_f['v1']

In [ ]:
scale_fns_f['v1']

In [ ]:
samples_out = []
px_out = []

feed_dict = {
    x: None,
    beta: 1.,
    net_weights[0]: 1.,
    net_weights[1]: 1.,
    net_weights[2]: 1.,
    train_phase: False
}
#samples = model.distribution.get_samples(model.num_samples)
samples = np.random.randn(*model.x.shape)
num_steps = 5000
for i in range(num_steps):
    t0 = time.time()
    feed_dict.update({
        x: samples,
    })
    samples, px_ = sess.run([x_out, px], feed_dict=feed_dict)
    _px = np.mean(px_)
    samples_out.append(samples)
    px_out.append(_px)
    print(f'{i}/{num_steps}, px: {_px:^.4g}, t/step: {time.time() - t0:^.4g}')

samples_out = np.array(samples_out)
all_samples = samples_out.reshape((-1, 2))
x_, y_ = all_samples.T
xmean, ymean = x_.mean(), y_.mean()

In [ ]:
samples_out_hmc = []
px_out_hmc = []

feed_dict = {
    model.x: None,
    model.beta: 1.,
    model.net_weights[0]: 0.,
    model.net_weights[1]: 0.,
    model.net_weights[2]: 0.,
    model.train_phase: False
}
samples = model.distribution.get_samples(model.num_samples)
num_steps = 5000
for i in range(num_steps):
    t0 = time.time()
    feed_dict.update({
        model.x: samples,
    })
    samples, px = sess.run([model.x_out, model.px], feed_dict=feed_dict)
    _px = np.mean(px)
    samples_out_hmc.append(samples)
    px_out_hmc.append(_px)
    print(f'{i}/{num_steps}, px: {_px:^.4g}, t/step: {time.time() - t0:^.4g}')
    
samples_out_hmc = np.array(samples_out_hmc)
all_samples_hmc = samples_out_hmc.reshape((-1, 2))
x_hmc, y_hmc = all_samples_hmc.T
xmean_hmc, ymean_hmc = x_hmc.mean(), y_hmc.mean()

In [ ]:
inference_dir = os.path.join(model.log_dir, 'inference')
io.check_else_make_dir(inference_dir)

l2hmc_shape_str = f'L2HMC samples.shape: {samples_out.shape}\n'
hmc_shape_str = f'HMC samples.shape: {samples_out_hmc.shape}\n'
l2hmc_str = f'L2HMC Averages (x_avg, y_avg): ({xmean:.5g}, {ymean:.5g})\n'
hmc_str = f'HMC Averages (x_avg, y_avg): ({xmean_hmc:.5g}, {ymean_hmc:.5g})\n'

print(l2hmc_shape_str + l2hmc_str)
print('\n')
print(hmc_shape_str + hmc_str)

samples_out_file = os.path.join(inference_dir, 'inference_samples.pkl')
px_out_file = os.path.join(inference_dir, 'inference_probs.pkl')
with open(samples_out_file, 'wb') as f:
    pickle.dump(samples_out_file, f)
with open(px_out_file, 'wb') as f:
    pickle.dump(px_out, f)
    
samples_out_file_hmc = os.path.join(inference_dir, 'hmc_samples.pkl')
px_out_file_hmc = os.path.join(inference_dir, 'hmc_probs.pkl')
with open(samples_out_file_hmc, 'wb') as f:
    pickle.dump(samples_out_hmc, f)
with open(px_out_file_hmc, 'wb') as f:
    pickle.dump(px_out_hmc, f)

inference_means_file = os.path.join(inference_dir, 'inference_means.txt')
with open(inference_means_file, 'w') as f:
    _ = f.write(l2hmc_shape_str)
    _ = f.write(l2hmc_str)
    _ = f.write(20 * '-' + '\n')
    _ = f.write(hmc_shape_str)
    _ = f.write(hmc_str)

In [ ]:
from plotters.plot_utils import gmm_plot

figs_dir = os.path.join(model.log_dir, 'figures')
io.check_else_make_dir(figs_dir)

l2hmc_kwargs = {
    'nrows': 4,
    'ncols': 4,
    'num_points': 500,
    'out_file': None,
    'title': 'L2HMC (Gaussian losss)',
    'out_file': os.path.join(figs_dir, 'inference_plot.pdf')
}
fig, axes = gmm_plot(model.distribution, samples_out, **l2hmc_kwargs)

In [ ]:
axes[0, 0].set_yticks(model.distribution.mus[:, 1])
axes[0, 0].set_yticks(model.distribution.mus[:, 1])

In [ ]:
l2hmc_kwargs = {
    'nrows': 4,
    'ncols': 4,
    'num_points': 500,
    'out_file': None,
    'title': 'L2HMC (Gaussian losss)',
    'out_file': os.path.join(gaussian_dir, 'figures',
                             'l2hmc_chains_5000_eps0144.pdf')
    
}
fig_g, axes_g = inference_plot(model, samples_out_g, **l2hmc_kwargs)

l2hmc_kwargs = {
    'nrows': 3,
    'ncols': 3,
    'num_points': 500,
    'out_file': None,
    'title': 'L2HMC (ESJD losss)'
}
fig, axes = inference_plot(model, samples_out, **l2hmc_kwargs)

In [ ]:
#target_samples = model.distribution.get_samples(5000)
samples_out = np.array(samples_out)
all_samples = samples_out.reshape((-1, 2))
figs_dir = os.path.join(model.log_dir, 'figures')
io.check_else_make_dir(figs_dir)
#x_lims = 1.5 * np.array([-0.275, 1.52])
#y_lims = 1.5 * np.array([-0.7, 1.35])
nrows = ncols = 4
fig, axes = plt.subplots(nrows=nrows, ncols=ncols)#, sharex=True, sharey=True)
#x_lims=[-0.5, 1.6]
#y_lims=[-1., 1.45]
#plt.subplots_adjust(hspace=0.25, wspace=0.25)
xmin = ymin = -1.5
xmax = ymax = 2.0
x_lims = [xmin, xmax]
y_lims = [ymin, ymax]

_idx = 0
for idx in range(nrows):
    for jdx in range(ncols):
        _ = plot_gaussian_contours(mus, covs,
                                   x_lims=x_lims,
                                   y_lims=y_lims,
                                   res=200,
                                   ax=axes[idx, jdx])
        _ = axes[idx, jdx].plot(samples_out[-1000:, _idx, 0],
                                samples_out[-1000:, _idx, 1],
                                marker=',', color='gray', alpha=0.4, ls='-')
        _ = axes[idx, jdx].plot(samples_out[-1000:, _idx, 0],
                                samples_out[-1000:, _idx, 1],
                                marker=',', color='k', alpha=0.6, ls='')
        _ = axes[idx, jdx].axis('equal')
        _ = axes[idx, jdx].set_xticks([])
        _ = axes[idx, jdx].set_yticks([])
        _idx += 1
        
_ = axes[0, 0].set_yticks(model.distribution.mus[0])
_ = axes[0, 0].set_yticklabels([str(i) for i in model.distribution.mus[0]])
_ = axes[-1, -1].set_xticks(model.distribution.mus[1])
_ = axes[-1, -1].set_xticklabels([str(i) for i in model.distribution.mus[1]])
_ = fig.suptitle('L2HMC Chains (standard loss)')
        
N = samples_out_hmc.shape[0]
eps_np = sess.run(model.dynamics.eps)
eps_str = f'{eps_np:.3g}'.replace('.', '')
out_file = os.path.join(figs_dir, f'l2hmc_chains_{N}_eps{eps_str}.pdf')
io.log(f'Saving figure to: {out_file}.') 
fig.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
#target_samples = model.distribution.get_samples(5000)
figs_dir = os.path.join(model.log_dir, 'figures')
io.check_else_make_dir(figs_dir)
x_lims = 1.5 * np.array([-0.275, 1.52])
y_lims = 1.5 * np.array([-0.7, 1.35])
#x_lims=[-0.35, 1.55]
#y_lims=[-0.7, 1.35]
nrows = ncols = 4
fig, axes = plt.subplots(nrows=nrows, ncols=ncols)#, sharex=True, sharey=True)
#_ = plt.subplots_adjust(hspace=0.25, wspace=0.25)
_idx = 0
for idx in range(nrows):
    for jdx in range(ncols):
        _ = plot_gaussian_contours(mus, covs,
                                   x_lims=x_lims,
                                   y_lims=y_lims,
                                   res=200,
                                   ax=axes[idx, jdx])
        _ = axes[idx, jdx].plot(samples_out_hmc[-1000:, _idx, 0],
                                samples_out_hmc[-1000:, _idx, 1],
                                marker=',', color='gray', alpha=0.4, ls='-')
        _ = axes[idx, jdx].plot(samples_out_hmc[-1000:, _idx, 0],
                                samples_out_hmc[-1000:, _idx, 1],
                                marker=',', color='k', alpha=0.6, ls='')
        _ = axes[idx, jdx].axis('equal')
        _ = axes[idx, jdx].set_xticks([])
        _ = axes[idx, jdx].set_yticks([])
        _idx += 1
        
_ = axes[0, 0].set_yticks(model.distribution.mus[0])
_ = axes[0, 0].set_yticklabels([str(i) for i in model.distribution.mus[0]])
_ = axes[-1, -1].set_xticks(model.distribution.mus[1])
_ = axes[-1, -1].set_xticklabels([str(i) for i in model.distribution.mus[1]])
_ = fig.suptitle('HMC Chains (standard loss)')
# _ = fig.tight_layout()

N = samples_out_hmc.shape[0]
eps_np = sess.run(model.dynamics.eps)
eps_str = f'{eps_np:.3g}'.replace('.', '')
out_file = os.path.join(figs_dir, f'hmc_chains_{N}_eps{eps_str}.pdf')
io.log(f'Saving figure to: {out_file}.') 
_ = fig.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
g_dir = '../../gmm_logs/2019_09_12/2019_09_12_1743_gaussian_loss/'
gaussian_dir = os.path.join(*g_dir.split('/'))
px_out_file_g = os.path.join(gaussian_dir, 'px_out.pkl')
samples_out_file_g = os.path.join(gaussian_dir, 'samples_out.pkl')

with open(px_out_file_g, 'rb') as f:
    px_out_g = pickle.load(f)
    
with open(samples_out_file_g, 'rb') as f:
    samples_out_g = pickle.load(f)
    

In [ ]:
px_out_g  = np.array(px_out_g)
samples_out_g = np.array(samples_out_g)
px_out_g.shape
samples_out_g.shape

print(f'GAUSSIAN LOSS:\n')
print(f' accept_prob:\n ')
print(f'  {px_out_g[:50]}\n')
print(f' avg. accept_prob:\n')
print(f'  {np.mean(px_out_g):.5g}')

In [ ]:
print(f'GAUSSIAN LOSS:\n')
print(f' accept_prob:\n ')
print(f'  {px_out_g[:50]}\n')
print(f' avg. accept_prob:\n')
print(f'  {np.mean(px_out_g):.5g}')

In [ ]:
print(f' accept_prob:\n ')
print(f'  {px_out[:50]}\n')
print(f' avg. accept_prob:  {np.mean(px_out):.5g}')

In [ ]:
print(f'accept_prob (gaussian_loss):\n {px_out_g[:50]}')

In [ ]:
print(f'accept_prob (standard loss):\n {px_out[:50]}')

In [ ]:
samples_out_file = os.path.join(model.log_dir, 'samples_out.pkl')
samples_out_hmc_file = os.path.join(model.log_dir, 'samples_out_hmc.pkl')
px_out_file = os.path.join(model.log_dir, 'px_out.pkl')
px_out_hmc_file = os.path.join(model.log_dir, 'px_out_hmc.pkl')

with open(samples_out_file, 'wb') as f:
    pickle.dump(samples_out, f)
with open(px_out_file, 'wb') as f:
    pickle.dump(px_out, f)
    
with open(samples_out_hmc_file, 'wb') as f:
    pickle.dump(samples_out_hmc, f)
with open(px_out_hmc_file, 'wb') as f:
    pickle.dump(px_out_hmc, f)

In [ ]:
import numpy as np

with open(samples_out_file, 'rb') as f:
    samples_in = pickle.load(f)
with open(px_out_file, 'rb') as f:
    px_in = pickle.load(f)
    
with open(samples_out_hmc_file, 'rb') as f:
    samples_in_hmc = pickle.load(f)
with open(px_out_hmc_file, 'rb') as f:
    px_in_hmc = pickle.load(f)
   
np.allclose(samples_out, samples_in)
np.allclose(px_out, px_in)

np.allclose(samples_out_hmc, samples_in_hmc)
np.allclose(px_in_hmc, px_out_hmc)

In [ ]:
#target_samples = model.distribution.get_samples(5000)
samples_out = np.array(samples_out)
all_samples = samples_out.reshape((-1, 2))
figs_dir = os.path.join(model.log_dir, 'figures')
io.check_else_make_dir(figs_dir)
x_lims = 1.5 * np.array([-0.275, 1.52])
y_lims = 1.5 * np.array([-0.7, 1.35])
nrows = ncols = 2
fig, axes = plt.subplots(nrows=nrows, ncols=ncols)#, sharex=True, sharey=True)
#x_lims=[-0.5, 1.6]
#y_lims=[-1., 1.45]
#plt.subplots_adjust(hspace=0.25, wspace=0.25)
_idx = 0
for idx in range(nrows):
    for jdx in range(ncols):
        _ = plot_gaussian_contours(mus, covs,
                                   x_lims=x_lims,
                                   y_lims=y_lims,
                                   res=200,
                                   ax=axes[idx, jdx])
        _ = axes[idx, jdx].plot(samples_out[:50, _idx, 0],
                                samples_out[:50, _idx, 1],
                                marker=',', color='gray', alpha=0.4, ls='-')
        _ = axes[idx, jdx].plot(samples_out[:50, _idx, 0],
                                samples_out[:50, _idx, 1],
                                marker=',', color='k', alpha=0.6, ls='')
        _ = axes[idx, jdx].axis('equal')
        _ = axes[idx, jdx].set_xticks([])
        _ = axes[idx, jdx].set_yticks([])
        _idx += 1
        
_ = axes[0, 0].set_yticks(model.distribution.mus[0])
_ = axes[0, 0].set_yticklabels([str(i) for i in model.distribution.mus[0]])
_ = axes[-1, -1].set_xticks(model.distribution.mus[1])
_ = axes[-1, -1].set_xticklabels([str(i) for i in model.distribution.mus[1]])
_ = fig.suptitle('L2HMC Chains (ESJD loss)')

In [ ]:
from scipy.misc import logsumexp


def quadratic_gaussian_np(x, mu, S):
    try:
        return np.diag(0.5 * np.matmul(
            np.matmul(x - mu, S),
            np.transpose(x - mu)
        ))
    except:
        return np.


def minus_log_likelihood_np(distribution, x):
    V = np.concatenate([np.expand_dims(
        -quadratic_gaussian_np(x, distribution.mus[i],
                               distribution.i_sigmas[i])
        + np.log(distribution.constants[i]), axis=1
    ) for i in range(distribution.nb_mixtures)], axis=1)

    return - logsumexp(V, axis=1), V  # using scipy.misc.logsumexp

In [ ]:
rand_samps = model.distribution.get_samples(10)
rand_samps[:5]
XY.shape

In [ ]:
x_lims = 1.5 * np.array([-0.35, 1.60])
y_lims = 1.5 * np.array([-0.60, 1.35])


x = np.linspace(x_lims[0], x_lims[1], 30)
y = np.linspace(y_lims[0], y_lims[1], 30)

X, Y = np.meshgrid(x, y)

In [ ]:
rand_samps[:5]

In [ ]:
XY.shape

In [ ]:
Z = minus_log_likelihood_np(model.distribution, (X, Y))

In [ ]:
XY = np.array([X, Y])
XY.shape

In [ ]:
X.shape

In [ ]:
Z = minus_log_likelihood_np()

In [ ]:
X[:5], X.shape
Y[:5], Y.shape

In [ ]:
def f(x, y):
    return np.sin(np.sqrt(x ** 2 + y ** 2))

x = np.linspace(-6, 6, 30)
y = np.linspace(-6, 6, 30)

X, Y = np.meshgrid(x, y)
Z = f(X, Y)

In [ ]:
def U(distribution, x, y):
    c0 = distribution.constants[0]
    gauss0 = 
    - np.log()

In [ ]:
def gaussian_np(x, mu, S):
def U(x, y):
    return np.log()

In [ ]:
q = rand_samps[0].dot(rand_samps[0].T)
q

In [ ]:
gauss_np = quadratic_gaussian_np(rand_samps[0],
                                 model.distribution.mus[0],
                                 model.distribution.i_sigmas[0])
gauss_np

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.contour3D(X, Y, Z, 50, cmap='binary')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z');

In [ ]:
#target_samples = model.distribution.get_samples(5000)
figs_dir = os.path.join(model.log_dir, 'figures')
io.check_else_make_dir(figs_dir)
x_lims = 1.5 * np.array([-0.275, 1.52])
y_lims = 1.5 * np.array([-0.7, 1.35])
#x_lims=[-0.35, 1.55]
#y_lims=[-0.7, 1.35]
nrows = ncols = 2
fig, axes = plt.subplots(nrows=nrows, ncols=ncols)#, sharex=True, sharey=True)
#_ = plt.subplots_adjust(hspace=0.25, wspace=0.25)
_idx = 0
for idx in range(nrows):
    for jdx in range(ncols):
        _ = plot_gaussian_contours(mus, covs,
                                   x_lims=x_lims,
                                   y_lims=y_lims,
                                   res=200,
                                   ax=axes[idx, jdx])
        _ = axes[idx, jdx].plot(samples_out_hmc[:100, _idx, 0],
                                samples_out_hmc[:100, _idx, 1],
                                marker=',', color='gray', alpha=0.4, ls='-')
        _ = axes[idx, jdx].plot(samples_out_hmc[:100, _idx, 0],
                                samples_out_hmc[:100, _idx, 1],
                                marker=',', color='k', alpha=0.6, ls='')
        _ = axes[idx, jdx].axis('equal')
        _ = axes[idx, jdx].set_xticks([])
        _ = axes[idx, jdx].set_yticks([])
        _idx += 1
        
_ = axes[0, 0].set_yticks(model.distribution.mus[0])
_ = axes[0, 0].set_yticklabels([str(i) for i in model.distribution.mus[0]])
_ = axes[-1, -1].set_xticks(model.distribution.mus[1])
_ = axes[-1, -1].set_xticklabels([str(i) for i in model.distribution.mus[1]])
_ = fig.suptitle('HMC Chains (ESJD loss)')
# _ = fig.tight_layout()

In [ ]:
def inference_plot(model, samples, **kwargs):
    nrows = kwargs.get('nrows', 3)
    ncols = kwargs.get('ncols', 3)
    out_file = kwargs.get('out_file', None)
    title = kwargs.get('title', None)
    num_points = kwargs.get('num_points', 2000)
    
    mus = model.distribution.mus
    sigmas = model.distribution.sigmas
    
    
    #cov_max = np.max(model.distribution.sigmas)
    #x_min = np.min(np.min(model.distribution.mus[:, 0])) - 4 * cov_max
    #x_max = np.max(np.max(model.distribution.mus[:, 0])) + 4 * cov_max
    #y_min = np.min(np.min(model.distribution.mus[:, 1])) - 4 * cov_max
    #y_max = np.max(np.max(model.distribution.mus[:, 1])) + 4 * cov_max
    
    #x_lims = np.array([x_min, x_max])
    #y_lims = np.array([y_min, y_max])
    
    x_lims = 1.5 * np.array([-0.275, 1.52])
    y_lims = 1.5 * np.array([-0.7, 1.35])
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols)
    _idx = 0
    for idx in range(nrows):
        for jdx in range(ncols):
            ax = axes[idx, jdx]
            _ = plot_gaussian_contours(mus, sigmas,
                                       x_lims=x_lims,
                                       y_lims=y_lims,
                                       res=200,
                                       ax=ax)
            _ = ax.plot(samples[:num_points, _idx, 0],
                        samples[:num_points, _idx, 1],
                        marker=',', ls='-',  color='gray', alpha=0.4)
            _ = ax.plot(samples[:num_points, _idx, 0],
                        samples[:num_points, _idx, 1],
                        marker=',', ls='', color='k', alpha=0.6)
            _ = axes[idx, jdx].axis('equal')
            _ = axes[idx, jdx].set_xticks([])
            _ = axes[idx, jdx].set_yticks([])
            _idx += 1

    _ = axes[0, 0].set_yticks(mus[0])
    _ = axes[0, 0].set_yticklabels([str(i) for i in mus[0]])
    _ = axes[-1, -1].set_xticks(mus[1])
    _ = axes[-1, -1].set_xticklabels([str(i) for i in mus[1]])
    if title is not None:
        _ = fig.suptitle(title)
    if out_file is not None:
        print(f'Saving figure to: {out_file}.')
        plt.savefig(out_file, dpi=400, bbox_inches='tight')
        
    return fig, axes

In [ ]:
l2hmc_kwargs = {
    'nrows': 4,
    'ncols': 4,
    'num_points': 500,
    'out_file': None,
    'title': 'L2HMC (Gaussian losss)',
    'out_file': os.path.join(gaussian_dir, 'figures',
                             'l2hmc_chains_5000_eps0144.pdf')
    
}
fig_g, axes_g = inference_plot(model, samples_out_g, **l2hmc_kwargs)

l2hmc_kwargs = {
    'nrows': 3,
    'ncols': 3,
    'num_points': 500,
    'out_file': None,
    'title': 'L2HMC (ESJD losss)'
}
fig, axes = inference_plot(model, samples_out, **l2hmc_kwargs)

# OLD

In [ ]:
from collections import namedtuple

def esjd(x1, x2, prob):
    """Expected squared jump distance (ESJD)."""
    return prob * np.sum(np.square(x1 - x2), axis=1) + 1e-4


x_dynamics = model._apply_transition(model.x, model.beta,
                                    model.net_weights,
                                    model.train_phase,
                                    save_lf=model.save_lf)

z_dynamics = model._apply_transition(model.z, model.beta,
                                    model.net_weights,
                                    model.train_phase,
                                    save_lf=model.save_lf)

In [ ]:
x_dynamics_, z_dynamics_ = sess.run([x_dynamics, z_dynamics],
                                    feed_dict=feed_dict)

In [ ]:
ls = getattr(model, 'loss_scale', 0.1)

esjd_x = esjd(x_dynamics_['x_in'],
              x_dynamics_['x_proposed'],
              x_dynamics_['accept_prob'])
esjd_z = esjd(z_dynamics_['x_in'],
              z_dynamics_['x_proposed'],
              z_dynamics_['accept_prob'])

x_loss = ls * np.mean(1. / esjd_x) - np.mean(esjd_x) / ls
z_loss = ls * np.mean(1. / esjd_z) - np.mean(esjd_z) / ls
loss = x_loss + z_loss
loss

In [ ]:
avg_esjd = (np.mean(esjd_x) + np.mean(esjd_z)) / 2
avg_esjd

In [ ]:
eps_np = sess.run(model.dynamics.eps)
md_dist = eps_np * model.num_steps
md_dist

In [ ]:
esjd_x.shape

In [ ]:
x_gaussian = np.exp(-0.5 * (esjd_x - md_dist)**2) / np.sqrt(2*np.pi) + 1e-4
z_gaussian = np.exp(-0.5 * (esjd_z - md_dist)**2) / np.sqrt(2*np.pi) + 1e-4

x_gauss_loss = - ls * np.mean(1. / x_gaussian) + np.mean(x_gaussian) / ls
z_gauss_loss = - ls * np.mean(1. / z_gaussian) + np.mean(z_gaussian) / ls
gauss_loss = x_gauss_loss + z_gauss_loss
gauss_loss

In [ ]:
esjd_diff = esjd_x - md_dist
esjd_diff[:10]

In [ ]:
x_gaussian = np.exp(-(esjd_diff**2) / 2) / np.sqrt(2*np.pi)
x_gaussian[:10]

In [ ]:
model.eps

In [ ]:
lambda_x_ = avg_diff_np(xdyn_['x_in'],
                        xdyn_['x_proposed'],
                        xdyn_['accept_prob'])
qg_x_ = np.exp(np.square(lambda_x_) / 2)
xl_ = np.mean(qg_x_) / ls
xl_inv_ = np.mean(1. / qg_x_) * ls
x_loss_ = xl_inv_ - xl_

In [ ]:
qg_x = tf.exp(tf.square(lambda_x) / 2)
xl = tf.reduce_mean(qg_x) / ls
xl_inv = tf.reduce_mean(1. / qg_x) * ls
x_loss = xl_inv - xl

In [ ]:
from utils.distributions import quadratic_gaussian

def avg_diff(x1, x2, prob):
    return prob * tf.reduce_sum(model.metric_fn(x1, x2), axis=1) + 1e-4


_shape = model.distribution.i_sigmas[0].shape
S = np.ones(_shape) / 2.

In [ ]:
lambda_x = avg_diff(x_data.x_in, x_data.x_proposed, x_data.prob)

In [ ]:
ls = getattr(model, 'loss_scale', 0.1)

qg_x = tf.exp(tf.square(lambda_x) / 2)
xl = tf.reduce_mean(qg_x) / ls
xl_inv = tf.reduce_mean(1. / qg_x) * ls
x_loss = xl_inv - xl

In [ ]:
samples_init = np.random.randn(*model.x.shape)
feed_dict = {
    model.x: samples_init,
    model.beta: 1.,
    model.net_weights[0]: 1.,
    model.net_weights[1]: 1.,
    model.net_weights[2]: 1.,
    model.train_phase: False
}

ops = [
    lambda_x,
    qg_x,
    xl,
    xl_inv,
    x_loss
]

outputs = sess.run(ops, feed_dict=feed_dict)

In [ ]:
lambda_x_, qg_x_, xl_, xl_inv_, x_loss_ = ops

In [ ]:
lambda_x_.shape

In [ ]:
lambda_z = avg_diff(z_data.x_in, z_data.x_proposed, z_data.prob)
qg_z = quadratic_gaussian(lambda_z,
                          mu=np.zeros(lambda_x.shape),
                          S=np.ones(lambda_x.shape))
    
    

In [ ]:
_shape = model.distribution.i_sigmas[0].shape
_shape

In [ ]:
S = np.ones(_shape)
S.shape

In [ ]:
model.distribution.i_sigmas[0]

In [ ]:
model.distribution.mus[0]

In [ ]:
lambda_x

In [ ]:
from scipy.stats import sem

# NOTE: samples_out.shape = (chain_length, num_chains, 2)
#    Where chain_length = number of configurations output 
#    from MH Accept/Reject (using the trained sampler)
print(f'samples_out.shape: {samples_out.shape}\n\n')
samples_avg = samples_out.mean(axis=0) # get average (x, y) for each chain

# Avg `x`, `y` coordinates for each
# of the `num_chains` chains ran in parallel
_x, _y = samples_avg.T

# standard error of the mean
_x_err = sem(_x)  
_y_err =sem(_y)

print(f'avg_x: {_x.mean():.4g} +/- {_x_err:.4g}')
print(f'avg_y: {_y.mean():.4g} +/- {_y_err:.4g}')

In [ ]:
mean_obs = np.array([_x.mean(), _y.mean()])
mean_true = np.array([0.5, 0.5])
np.linalg.norm(mean_obs - mean_true)

In [ ]:
x_in = tf.random_normal(model.x.shape, dtype=TF_FLOAT, seed=GLOBAL_SEED,
                        name='x_reverse_check')
v_in = tf.random_normal(model.x.shape, dtype=TF_FLOAT, seed=GLOBAL_SEED,
                        name='v_reverse_check')

dynamics_check = model.dynamics._check_reversibility(x_in,
                                                     v_in,
                                                     model.beta,
                                                     model.net_weights,
                                                     model.train_phase)
xf = dynamics_check['xf']
vf = dynamics_check['vf']
xb = dynamics_check['xb']
vb = dynamics_check['vb']

feed_dict = {
    model.beta: 1.,
    model.net_weights[0]: 1.,
    model.net_weights[1]: 1.,
    model.net_weights[2]: 1.,
    model.train_phase: False
}

xf_, vf_, xb_, vb_, x_in_, v_in_ = sess.run([xf, vf, xb, vb, x_in, v_in],
                                            feed_dict=feed_dict)

# backward(forward(x_in)) ~ x_in
x_diff = np.sum((x_in_ - xb_).T.dot(x_in_ - xb_))
v_diff = np.sum((v_in_ - vb_).T.dot(v_in_ - vb_))

x_diff, v_diff

## OLD

In [ ]:
chain_length, num_chains, _ = samples_out.shape
fig, ax = plt.subplots()
_ = ax.plot(target_samples[:, 0], target_samples[:, 1], marker='.', alpha=0.4, ls='', color='lightgray')
_ = ax.plot(samples_out[-200:, idx, 0], samples_out[-200:, idx, 1], marker='.', alpha=0.4, ls='--', color='gray')
_ = ax.plot(_x.mean(), _y.mean(), marker='.', markersize=10, ls='', color='r', label=f'Avg. over {num_chains} ran for {chain_length} steps')
_ = ax.plot(0.5, 0.5, marker='s', markersize=10, ls='', color='blue', label=f'True avg.', fillstyle='none')
_ = ax.legend(loc='best')

#print(f'<x> = {_x.mean():.3g} +/- {_x_err:.3g}')
#print(f'<y> = {_y.mean():.3g} +/- {_y_err:.3g}')

In [ ]:
_x, _y = samples_avg0.T

In [ ]:
_x.shape

In [ ]:
fig, ax = plt.subplots()
ax.plot(samples_avg0[:, 0], samples_avg0[:, 1], marker='.', ls='', color='k')
ax.plot(_x.mean(), _y.mean(), marker='X', ls='', color='r', markersize=5.)
ax.set_xlim((0, 1))
ax.set_ylim((0, 1))

In [ ]:
_x.mean(), _y.mean()

In [ ]:
gmm_model.means

In [ ]:
target_x, target_y = gmm_model.means.T

In [ ]:
target_x.mean(), target_y.mean()

In [ ]:
sample_idx = 0
#data = np.random.multivariate_normal([0, 0], [[1, 0.5], [0.5, 3]], 200)
data = samples_out[:, sample_idx]
x, y = data.T

In [ ]:
data = np.random.multivariate_normal([0, 0], [[1, 0.5], [0.5, 3]], 200)
data.shape

In [ ]:
means1, means2 = gmm_model.means
means1

In [ ]:
gmm_model.means

In [ ]:
# Create a figure with 6 plot areas
from scipy.stats import kde

means1, means2 = gmm_model.means
    
def make_2d_density_plots(chain, name='', out_file=None):
    print(f'Chain: {name}')
    x, y = chain.T
    #data = samples_out[:, idx]
    #x, y = data.T
    
    fig, axes = plt.subplots(ncols=6, nrows=1, figsize=(21, 5))

    # Everything sarts with a Scatterplot
    _ = axes[0].set_title('Scatterplot')
    _ = axes[0].plot(x, y, 'ko')
    _ = axes[0].plot(means1[0], means1[1], marker='x', markersize=5., color='red')
    _ = axes[0].plot(means2[0], means2[1], marker='x', markersize=5., color='red')
    # As you can see there is a lot of overplottin here!

    # Thus we can cut the plotting window in several hexbins
    nbins = 20
    _ = axes[1].set_title('Hexbin')
    _ = axes[1].hexbin(x, y, gridsize=nbins, cmap=plt.cm.BuGn_r)
    _ = axes[1].plot(means1[0], means1[1], marker='x', markersize=5., color='red')
    _ = axes[1].plot(means2[0], means2[1], marker='x', markersize=5., color='red')

    # 2D Histogram
    _ = axes[2].set_title('2D Histogram')
    _ = axes[2].hist2d(x, y, bins=nbins, cmap=plt.cm.BuGn_r)
    _ = axes[2].plot(means1[0], means1[1], marker='x', markersize=5., color='red')
    _ = axes[2].plot(means2[0], means2[1], marker='x', markersize=5., color='red')

    # Evaluate a gaussian kde on a regular grid of nbins x nbins over data extents
    k = kde.gaussian_kde(chain.T)
    xi, yi = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
    zi = k(np.vstack([xi.flatten(), yi.flatten()]))

    # plot a density
    _ = axes[3].set_title('Calculate Gaussian KDE')
    _ = axes[3].pcolormesh(xi, yi, zi.reshape(xi.shape), cmap=plt.cm.BuGn_r)
    _ = axes[3].plot(means1[0], means1[1], marker='x', markersize=5., color='red')
    _ = axes[3].plot(means2[0], means2[1], marker='x', markersize=5., color='red')

    # add shading
    _ = axes[4].set_title('2D Density with shading')
    _ = axes[4].pcolormesh(xi, yi, zi.reshape(xi.shape), shading='gouraud', cmap=plt.cm.BuGn_r)
    _ = axes[4].plot(means1[0], means1[1], marker='x', markersize=5., color='red')
    _ = axes[4].plot(means2[0], means2[1], marker='x', markersize=5., color='red')

    # contour
    _ = axes[5].set_title('Contour')
    _ = axes[5].pcolormesh(xi, yi, zi.reshape(xi.shape), shading='gouraud', cmap=plt.cm.BuGn_r)
    _ = axes[5].contour(xi, yi, zi.reshape(xi.shape) )
    _ = axes[5].plot(means1[0], means1[1], marker='x', markersize=5., color='red')
    _ = axes[5].plot(means2[0], means2[1], marker='x', markersize=5., color='red')
    _ = plt.suptitle(f'chain.shape: {chain.shape}', fontsize=16)
    
    if out_file is not None:
        print(f'Saving figure to: {out_file}')
        plt.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
all_samples = samples_out.reshape(-1, 2)
all_samples.shape

In [ ]:
out_dir = '../figures/gmm_figures/'
for idx in range(5):
    out_file = os.path.join(*out_dir.split('/'), f'gmm_density_plots_{idx}.pdf')
    _ = make_2d_density_plots(samples_out[:, idx], name=f'{idx} idx', out_file=out_file)
    
all_file = os.path.join(*out_dir.split('/'), 'gmm_density_plots_all_chains.pdf')
_ = make_2d_density_plots(all_samples, name='all chains', out_file=all_file)

target_file = os.path.join(*out_dir.split('/'), f'gmm_density_plots_target_dist.pdf')
_ = make_2d_density_plots(target_samples, name='target distribution', out_file=target_file)

In [ ]:
all_file = os.path.join(*out_dir.split('/'), 'gmm_density_plots_all_chains2.pdf')
_ = make_2d_density_plots(all_samples[-20000:], name='all chains2', out_file=all_file)

In [ ]:
from models.gmm_model import distribution_arr

x_dim = 2
sigma = 0.02
centers = 1
means = np.zeros((x_dim, x_dim), dtype=np.float32)
for i in range(x_dim):
    means[i::x_dim, i] = centers
cov_mtx = sigma * np.eye(x_dim).astype(np.float32)
covs = np.array([cov_mtx] * x_dim).astype(np.float32)
dist_arr = distribution_arr(x_dim, 2)
gmm_dist = GMM(means, covs, dist_arr)

gmm_mus = np.array(gmm_dist.mus)
gmm_diffs = gmm_mus[1:] - gmm_mus[:-1, :]
gmm_distances = [np.sqrt(np.dot(d, d.T)) for d in gmm_diffs]
gmm_distances

In [ ]:
gmm_samples = gmm_dist.get_samples(500)
fig, ax = plt.subplots()
ax.plot(gmm_samples[:,0], gmm_samples[:,1], marker='o', ls='', alpha=0.75)
plt.show()

In [ ]:
k()

In [ ]:
zi.shape

In [ ]:
xi.shape